In [ ]:
import mysql.connector
import pandas as pd

# Koneksi ke database
def establish_connection():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='data_smartphone',
        port=3306
    )

# Eksekusi query dan konversi ke DataFrame
def execute_query_to_dataframe(connection, query):
    try:
        cursor = connection.cursor(dictionary=True)
        cursor.execute(query)
        result = cursor.fetchall()
        return pd.DataFrame(result)
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        cursor.close()

# Contoh penggunaan
connection = establish_connection()
if connection.is_connected():
    print("Koneksi ke database berhasil")
    query = """
    SELECT
        data.toko as toko,
        data.Merek,
        data.Tipe,
        data.Bulan,
        data.tahun,
        data.Jumlah_Stok,
        data.Jumlah_Terjual,
        data.Harga_Satuan_Rp,
        data.Total_Penjualan_Rp,
        data_spesifikasi.Kamera_Utama_MP,
        data_spesifikasi.Kamera_Depan_MP,
        data_spesifikasi.RAM,
        data_spesifikasi.Memori_Internal,
        data_spesifikasi.Baterai_mAh,
        data_spesifikasi.Jenis_Layar
    FROM 
        data
    LEFT JOIN 
        data_spesifikasi 
    ON 
        data.Tipe = data_spesifikasi.Tipe
    WHERE 
        data.toko = 'Jaya Com'
    """
    df = execute_query_to_dataframe(connection, query)
    display(df.head())
else:
    print("No connection to the database.")


In [ ]:
import numpy as np

# Menangani nilai yang hilang
def handle_missing_values(df):
    df.fillna({'': 0, np.number: 0}, inplace=True)

# Kategorisasi persentase penjualan
def categorize_sales_percentage(percent_sold):
    if percent_sold < 10:
        return 'Sangat Rendah'
    elif percent_sold < 20:
        return 'Rendah'
    elif percent_sold < 50:
        return 'Cukup'
    elif percent_sold < 70:
        return 'Tinggi'
    else:
        return 'Sangat Tinggi'

# Persiapan DataFrame
if df is not None and not df.empty:
    columns_needed = ['Merek', 'Tipe', 'Jumlah_Stok', 'Jumlah_Terjual',
                      'Harga_Satuan_Rp', 'Total_Penjualan_Rp',
                      'Kamera_Utama_MP', 'Kamera_Depan_MP', 'RAM',
                      'Memori_Internal', 'Baterai_mAh', 'Jenis_Layar', 'Bulan', 'tahun']

    df_klastering = df[columns_needed].copy()
    df_klastering['Persentase_Jumlah_Terjual'] = (df_klastering['Jumlah_Terjual'] / df_klastering['Jumlah_Stok']) * 100

    handle_missing_values(df_klastering)
    df_klastering['Kategori_Penjualan'] = df_klastering['Persentase_Jumlah_Terjual'].apply(categorize_sales_percentage)

    display(df_klastering.head())
else:
    print("No data available for processing.")


In [ ]:
from sklearn.cluster import Birch

# Inisialisasi dan penerapan model BIRCH
birch_model = Birch(n_clusters=5, threshold=0.5)
df_features = df_klastering[['Jumlah_Stok', 'Jumlah_Terjual', 'Persentase_Jumlah_Terjual']]
birch_model.fit(df_features)

clusters = birch_model.predict(df_features)
df_klastering['Cluster'] = clusters
centroids = birch_model.subcluster_centers_

display(df_klastering.head())


In [ ]:
import plotly.graph_objs as go
import pandas as pd

def visualize_sales_distribution_combined(df):
    # Mapping warna untuk kategori penjualan
    color_mapping = {
        'Sangat Rendah': 'blue',
        'Rendah': 'green',
        'Cukup': 'gold',
        'Tinggi': 'salmon',
        'Sangat Tinggi': 'maroon'
    }
    
    # Menghitung frekuensi kategori penjualan per merek
    kategori_per_merek = df.groupby(['Merek', 'Kategori_Penjualan']).size().unstack(fill_value=0)

    # Hitung total produk per merek
    total_per_merek = kategori_per_merek.sum(axis=1)

    # Menyiapkan data untuk grafik batang
    bar_data = []
    for kategori in kategori_per_merek.columns:
        bar_data.append(
            go.Bar(
                name=kategori,
                x=kategori_per_merek.index,
                y=kategori_per_merek[kategori],
                marker_color=color_mapping[kategori],
                text=[f"{count} ({(count / total_per_merek[merek] * 100):.1f}%)" 
                      for merek, count in zip(kategori_per_merek.index, kategori_per_merek[kategori])],
                textposition='auto'
            )
        )

    # Membuat grafik batang
    fig = go.Figure(data=bar_data)

    # Update layout
    fig.update_layout(
        barmode='stack',
        title='Distribusi Kategori Penjualan per Merek Untuk Jaya Com',
        xaxis_title='Merek',
        yaxis_title='Jumlah Produk',
        height=900,
        # width=1000
    )

    fig.show()

    # Menyusun tabel distribusi per merek
    tabel_distribusi = []

    for merek in kategori_per_merek.index:
        for kategori in kategori_per_merek.columns:
            jumlah = kategori_per_merek.loc[merek, kategori]
            persentase = (jumlah / total_per_merek[merek]) * 100
            tabel_distribusi.append({
                'Merek': merek,
                'Kategori Penjualan': kategori,
                'Jumlah': jumlah,
                'Persentase': f"{persentase:.1f}%"
            })
    
    # Konversi list of dicts ke DataFrame
    tabel_distribusi_df = pd.DataFrame(tabel_distribusi)
    
    # Menampilkan tabel distribusi
    display(tabel_distribusi_df)

# Contoh penggunaan fungsi
visualize_sales_distribution_combined(df_klastering)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objs as go

def visualize_clusters_tsne(df):
    # Hanya pilih kolom yang relevan
    features = ['Jumlah_Stok', 'Jumlah_Terjual', 'Persentase_Jumlah_Terjual']
    df_numeric = df[features].copy()
    
    # Konversi kolom ke tipe numerik dan tangani nilai yang hilang
    for col in df_numeric.columns:
        df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')
    
    df_numeric = df_numeric.fillna(df_numeric.median())
    X_normalized = (df_numeric.values - df_numeric.values.mean(axis=0)) / df_numeric.values.std(axis=0)
    
    tsne = TSNE(n_components=2, random_state=42)
    X_tsne = tsne.fit_transform(X_normalized)
    
    df_plot = df[['Merek', 'Tipe', 'Kategori_Penjualan']].copy()
    df_plot['x'] = X_tsne[:, 0]
    df_plot['y'] = X_tsne[:, 1]
    
    color_map = {
        'Sangat Rendah': '#FF4136',  # Merah
        'Rendah': '#2ECC40',  # Hijau
        'Cukup': '#0074D9',  # Biru
        'Tinggi': '#FF851B',  # Oranye
        'Sangat Tinggi': '#B10DC9',  # Ungu
    }
    
    fig = go.Figure()

    # Plotkan setiap kategori penjualan
    for category in df_plot['Kategori_Penjualan'].unique():
        category_data = df_plot[df_plot['Kategori_Penjualan'] == category]
        fig.add_trace(go.Scatter(
            x=category_data['x'],
            y=category_data['y'],
            mode='markers',
            marker=dict(size=8, color=color_map.get(category, 'grey'), opacity=0.7),
            text=category_data['Merek'] + ' - ' + category_data['Tipe'] + '<br>Kategori: ' + category,
            hoverinfo='text',
            name=category
        ))

    fig.update_layout(
        title='Visualisasi t-SNE Berdasarkan Kategori Penjualan Jaya Com',
        xaxis_title='t-SNE Dimension 1',
        yaxis_title='t-SNE Dimension 2',
        legend_title='Kategori Penjualan',
        height=800,
        width=1000,
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(size=12)
    )

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

    fig.show()

    # Tampilkan statistik per kategori
    category_stats = df_plot.groupby('Kategori_Penjualan').agg({
        'Merek': 'count'
    }).rename(columns={'Merek': 'Jumlah Data'})
    
    display(category_stats)

# Contoh penggunaan fungsi
visualize_clusters_tsne(df_klastering)


In [ ]:
def visualize_clusters_3d(df, clusters):
    features = ['Jumlah_Stok', 'Jumlah_Terjual', 'Persentase_Jumlah_Terjual']
    df_numeric = df[features].copy()
    
    for col in df_numeric.columns:
        df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')
    
    df_numeric = df_numeric.fillna(df_numeric.median())
    X_normalized = (df_numeric.values - df_numeric.values.mean(axis=0)) / df_numeric.values.std(axis=0)
    
    tsne = TSNE(n_components=3, random_state=42)
    X_tsne = tsne.fit_transform(X_normalized)
    
    df_plot = df[['Merek', 'Tipe', 'Kategori_Penjualan']].copy()
    df_plot['x'] = X_tsne[:, 0]
    df_plot['y'] = X_tsne[:, 1]
    df_plot['z'] = X_tsne[:, 2]
    df_plot['Cluster'] = clusters
    
    color_map = {
        0: '#FF4136',  
        1: '#2ECC40',  
        2: '#0074D9',  
        3: '#FF851B',  
        4: '#B10DC9',  
    }
    
    fig = go.Figure()

    for cluster in range(5):  
        cluster_data = df_plot[df_plot['Cluster'] == cluster]
        fig.add_trace(go.Scatter3d(
            x=cluster_data['x'],
            y=cluster_data['y'],
            z=cluster_data['z'],
            mode='markers',
            marker=dict(size=4, color=color_map[cluster], opacity=0.7),
            text=cluster_data['Merek'] + ' - ' + cluster_data['Tipe'] + '<br>Kategori: ' + cluster_data['Kategori_Penjualan'],
            hoverinfo='text',
            name=f'Cluster {cluster}'
        ))

    fig.update_layout(
        title='Visualisasi 3D 5 Cluster Utama BIRCH menggunakan t-SNE',
        scene=dict(
            xaxis_title='t-SNE Dimension 1',
            yaxis_title='t-SNE Dimension 2',
            zaxis_title='t-SNE Dimension 3',
        ),
        legend_title='Clusters',
        height=800,
        width=1000,
        margin=dict(r=20, b=10, l=10, t=40)
    )

    fig.show()

    cluster_stats = df_plot.groupby('Cluster').agg({
        'Merek': 'count',
        'Kategori_Penjualan': lambda x: x.value_counts().index[0]
    }).rename(columns={'Merek': 'Jumlah Data', 'Kategori_Penjualan': 'Kategori Dominan'})
    
    display(cluster_stats)

visualize_clusters_3d(df_klastering, clusters)


In [ ]:
from sklearn.manifold import TSNE
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def visualize_tsne_by_brand(df, clusters):
    # Pilih fitur untuk t-SNE
    features = ['Jumlah_Stok', 'Jumlah_Terjual', 'Persentase_Jumlah_Terjual']
    df_numeric = df[features].copy()
    
    # Konversi kolom ke tipe numerik dan tangani nilai yang hilang
    for col in df_numeric.columns:
        df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')
    
    df_numeric = df_numeric.fillna(df_numeric.median())
    X_normalized = (df_numeric.values - df_numeric.values.mean(axis=0)) / df_numeric.values.std(axis=0)
    
    # Aplikasikan t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    X_tsne = tsne.fit_transform(X_normalized)
    
    df['x'] = X_tsne[:, 0]
    df['y'] = X_tsne[:, 1]
    df['Cluster'] = clusters
    
    # Dapatkan daftar merek
    merek_list = df['Merek'].unique()
    
    # Setup grid 5x2 untuk plot
    rows, cols = 2, 4
    fig = make_subplots(
        rows=rows, cols=cols, 
        subplot_titles=merek_list[:rows*cols],
        vertical_spacing=0.05,
        horizontal_spacing=0.05
    )

    # Warna kontras untuk klaster
    color_map = {
        0: '#FF4136',  # Merah
        1: '#2ECC40',  # Hijau
        2: '#0074D9',  # Biru
        3: '#FF851B',  # Oranye
        4: '#B10DC9',  # Ungu
    }

    # Tambahkan scatter plot untuk setiap merek
    for i, merek in enumerate(merek_list[:rows*cols]):
        df_merek = df[df['Merek'] == merek]
        fig.add_trace(
            go.Scatter(
                x=df_merek['x'], 
                y=df_merek['y'], 
                mode='markers', 
                marker=dict(size=6, color=[color_map[cluster] for cluster in df_merek['Cluster']], opacity=0.7),
                text=df_merek['Tipe'] + '<br>Kategori: ' + df_merek['Kategori_Penjualan'],
                hoverinfo='text',
                showlegend=False
            ),
            row=(i//cols)+1, col=(i%cols)+1
        )

    # Update layout untuk menambahkan judul dan memperbaiki tampilan
    fig.update_layout(
        title_text="Visualisasi t-SNE Berdasarkan Merek Jaya Com",
        height=600 * rows,  # Sesuaikan tinggi berdasarkan jumlah baris
        showlegend=False
    )

    # Tampilkan plot
    fig.show()

# Panggil fungsi visualisasi t-SNE berdasarkan merek
visualize_tsne_by_brand(df_klastering, clusters)
